### Contents

[Imports](#Imports)

[bottom](#bottom)

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pprint import pprint
import sys 
import json 
import itertools 
from tqdm.auto import tqdm
from datetime import datetime, date, timedelta
from load_dotenv import load_dotenv
load_dotenv() # OPENAI_API_KEY

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display, clear_output

/Users/antonandreytsev/llm_zoomcamp_venv3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
matplotlib.rcParams['figure.figsize'] = (7, 7)
sns.set_style('whitegrid')

---

In [4]:
import openai
from openai import OpenAI

In [5]:
# os.environ['OPENAI_API_KEY'] = 'xxxx'

In [13]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)
client

In [10]:
# # q = 'What is the capital of Russia?'
# q = "What is the purpose of human life?"
# response = client.chat.completions.create(
#     model="phi3",
#     messages=[{"role": "user", "content": q}]
# )
# response_content = response.choices[0].message.content
# print(response_content)

In [14]:
# q = "What is the purpose of human life?"
q = "What's the formula for energy?"
response = client.chat.completions.create(
    model="gemma:2b",
    messages=[{"role": "user", "content": q}],
    temperature=0.0,
)
response_content = response.choices[0].message.content
print(response_content)

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2mv^2**

**Potential energy (U)** is the energy an object possesses due to its position or configuration. It is calculated as the product of an object's mass, gravitational constant (g), and height or position above a reference point.

**U = mgh**

**Where:**

* **m** is the mass in kilograms (kg)
* **g** is the acceleration due to gravity in meters per second squared (m/s²)
* **h** is the height or position in meters (m)

The formula shows that energy can be expressed as the sum of kinetic and potential energy. The kinetic energy is a measure of the object's ability to do work, while the potential energy is a measur

In [24]:
response

ChatCompletion(id='chatcmpl-918', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's the formula for energy:\n\n**E = K + U**\n\nWhere:\n\n* **E** is the energy in joules (J)\n* **K** is the kinetic energy in joules (J)\n* **U** is the potential energy in joules (J)\n\n**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:\n\n**K = 1/2mv^2**\n\n**Potential energy (U)** is the energy an object possesses due to its position or configuration. It is calculated as the product of an object's mass, gravitational constant (g), and height or position above a reference point.\n\n**U = mgh**\n\n**Where:**\n\n* **m** is the mass in kilograms (kg)\n* **g** is the acceleration due to gravity in meters per second squared (m/s²)\n* **h** is the height or position in meters (m)\n\nThe formula shows that energy can be e

In [28]:
print(response.usage.completion_tokens)

283


### bottom